# USCFormer 

# Imports

In [30]:
# imports
import os
import random
import time
import torch
import math
import argparse
import numpy as np
import torch.nn as nn
import warnings
import torch.nn.functional
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from timm.models.layers import to_2tuple, trunc_normal_
from torchsummary import summary

# own files import
from datasets.train_data_functions import TrainData
from datasets.val_data_functions import ValData
from models import seg_network
from models.USCFormer import USCFormer
from models.SwinTransformer import SwinTransformerLayer
from loss import MSSSIM, CR
from utils import to_psnr, print_log, validation, adjust_learning_rate

# Global Parameters

In [31]:
plt.switch_backend('agg')

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

#train_data_dir = './data/cityscapes/train'
#val_data_dir = './data/cityscapes/val'

train_data_dir = '/home/yan/projects/xjm/TransDehaze/data/cityscapes/train'
val_data_dir = '/home/yan/projects/xjm/TransDehaze/data/cityscapes/val'

# --- Gpu device --- #
device_ids = [Id for Id in range(torch.cuda.device_count())]
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Hyper-Parameters

In [32]:
# --- Parse hyper-parameters  --- #
parser = argparse.ArgumentParser(description='Hyper-parameters for network')
parser.add_argument('-learning_rate', help='Set the learning rate', default=2e-4, type=float)
parser.add_argument('-crop_size', help='Set the crop_size', default=[256, 256], nargs='+', type=int)
parser.add_argument('-train_batch_size', help='Set the training batch size', default=4, type=int)
parser.add_argument('-epoch_start', help='Starting epoch number of the training', default=0, type=int)
parser.add_argument('-alpha_loss', help='Set the lambda in loss function', default=1, type=float)
parser.add_argument('-beta_loss', help='Set the lambda in loss function', default=0.02, type=float)
parser.add_argument('-gamma_loss', help='Set the lambda in loss function', default=0, type=float)
parser.add_argument('-val_batch_size', help='Set the validation/test batch size', default=1, type=int)
parser.add_argument('-exp_name', help='directory for saving the networks of the experiment', type=str,default='checkpoints')
parser.add_argument('-seed', help='set random seed', default=19, type=int)
parser.add_argument('-num_epochs', help='number of epochs', default=200, type=int)
parser.add_argument("--model", type=str, default='deeplabv3plus_mobilenet',
                    choices=['deeplabv3_resnet50', 'deeplabv3plus_resnet50',
                             'deeplabv3_resnet101', 'deeplabv3plus_resnet101',
                             'deeplabv3_mobilenet', 'deeplabv3plus_mobilenet'], help='model name')
parser.add_argument("--separable_conv", action='store_true', default=False,
                        help="apply separable conv to decoder and aspp")
parser.add_argument("--output_stride", type=int, default=16, choices=[8, 16])
parser.add_argument("--num_classes", type=int, default=19, help="num classer (default:19)")  #cityscapes默认19个类别
parser.add_argument("--ckpt", type=str, default="./snapshots/best_deeplabv3plus_mobilenet_cityscapes_os16.pth",
                    help="restore from checkpoint")

args = parser.parse_args(args=[])

learning_rate = args.learning_rate
crop_size = args.crop_size
train_batch_size = args.train_batch_size
epoch_start = args.epoch_start
alpha_loss = args.alpha_loss
beta_loss = args.beta_loss
gamma_loss = args.gamma_loss
val_batch_size = args.val_batch_size
exp_name = args.exp_name
num_epochs = args.num_epochs
models = args.model
separable_conv = args.separable_conv
output_stride = args.output_stride
num_classes = args.num_classes
ckpt = args.ckpt

print('--- Hyper-parameters for training ---')
print('learning_rate: {}\ncrop_size: {}\ntrain_batch_size: {}\nval_batch_size: {}\nalpha_loss: {}\nbeta_loss: {}\ngamma_loss: {}'.format(learning_rate, crop_size, train_batch_size, val_batch_size, alpha_loss, beta_loss, gamma_loss))

--- Hyper-parameters for training ---
learning_rate: 0.0002
crop_size: [256, 256]
train_batch_size: 4
val_batch_size: 1
alpha_loss: 1
beta_loss: 0.02
gamma_loss: 0


# Random Seed

In [33]:
#set seed
seed = args.seed
if seed is not None:
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    random.seed(seed) 
    print('Seed:\t{}'.format(seed))

Seed:	19


# Load Data

In [34]:
# --- Load training data and validation/test data --- #
lbl_train_data_loader = DataLoader(TrainData(crop_size, train_data_dir), batch_size=train_batch_size, shuffle=True, num_workers=8)
val_data_loader = DataLoader(ValData(val_data_dir), batch_size=val_batch_size, shuffle=False, num_workers=8)

# USCFormer Design

In [6]:
# Use ConvStream module to process local information of input image
class ConvStream4x(nn.Module):
    """
    Use ConvStream module to process local information of input image

    Input:
        - x: (B, 3, H, W)
    Output:
        - result: (B, 32, H, W)
    """
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size = 2, stride = 2, padding = 0, bias = False)
        self.gelu = nn.GELU()
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size = 2, stride = 2, padding = 0, bias = False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv3 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels)
        self.init_weight()

    def init_weight(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight)
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def forward(self, x):
        x = self.gelu(self.bn1(self.conv1(x)))
        x = self.bn2(self.conv2(x))
        result = self.bn3(self.conv3(x))
        return result

class ConvStream2x(nn.Module):
    """
    Use ConvStream module to process local information of input image

    Input:
        - x: (B, 3, H, W)
    Output:
        - result: (B, 32, H, W)
    """
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size = 2, stride = 2, padding = 0, bias = False)
        self.gelu = nn.GELU()
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size = 3, stride = 1, padding = 1, bias = False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv3 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels)
        self.init_weight()

    def init_weight(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight)
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def forward(self, x):
        x = self.gelu(self.bn1(self.conv1(x)))
        x = self.bn2(self.conv2(x))
        result = self.bn3(self.conv3(x))
        return result


In [7]:
#Image to Patch Embedding
class PatchEmbed(nn.Module):
    """ Image to Patch Embedding
    """

    def __init__(self, img_size=224, patch_size=7, stride=4, in_chans=3, embed_dim=768):
        super().__init__()
        img_size = to_2tuple(img_size)
        patch_size = to_2tuple(patch_size)

        self.img_size = img_size
        self.patch_size = patch_size
        self.H, self.W = img_size[0] // patch_size[0], img_size[1] // patch_size[1]
        self.num_patches = self.H * self.W
        self.proj = nn.Conv2d(in_chans, embed_dim, kernel_size=patch_size, stride=stride,
                              padding=(patch_size[0] // 2, patch_size[1] // 2))
        self.norm = nn.LayerNorm(embed_dim)

        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=.02)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)
        elif isinstance(m, nn.Conv2d):
            fan_out = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
            fan_out //= m.groups
            m.weight.data.normal_(0, math.sqrt(2.0 / fan_out))
            if m.bias is not None:
                m.bias.data.zero_()

    def forward(self, x):
        # pdb.set_trace()
        x = self.proj(x)
        B, C, H, W = x.shape
        x = x.flatten(2).transpose(1, 2)
        x = self.norm(x)
        x = x.reshape(B, H, W, -1).permute(0, 3, 1, 2).contiguous()
        return x, H, W

In [8]:
# The encoder of Transformer stem 
class EncoderTransformer(nn.Module):
    def __init__(self, img_size=224, in_chans=3, num_classes=1000, embed_dims=[96, 192, 384, 768],
                 num_heads=[3, 6, 12, 24], depths=[2, 2, 6, 2], window_size = 7):
        super().__init__()
        self.num_classes = num_classes
        self.depths = depths


        # Conv stem definitions
        self.Conv_stem1 = ConvStream4x(in_chans, embed_dims[0])
        self.Conv_stem2 = ConvStream2x(embed_dims[0], embed_dims[1])
        self.Conv_stem3 = ConvStream2x(embed_dims[1], embed_dims[2])
        self.Conv_stem4 = ConvStream2x(embed_dims[2], embed_dims[3])


        # patch embedding definitions
        self.patch_embed1 = PatchEmbed(img_size=img_size, patch_size=7, stride=4, in_chans=in_chans,
                                              embed_dim=embed_dims[0])
        self.patch_embed2 = PatchEmbed(img_size=img_size // 4, patch_size=3, stride=2, in_chans=embed_dims[0],
                                              embed_dim=embed_dims[1])
        self.patch_embed3 = PatchEmbed(img_size=img_size // 8, patch_size=3, stride=2, in_chans=embed_dims[1],
                                              embed_dim=embed_dims[2])
        self.patch_embed4 = PatchEmbed(img_size=img_size // 16, patch_size=3, stride=2, in_chans=embed_dims[2],
                                              embed_dim=embed_dims[3])

        self.block1 = nn.Sequential(*[
            SwinTransformerLayer(
                dim=embed_dims[0], num_heads=num_heads[0], window_size=window_size, shift_size=0 if (i % 2 == 0) else window_size // 2)
            for i in range(depths[0])])

        self.block2 = nn.Sequential(*[
            SwinTransformerLayer(
                dim=embed_dims[1], num_heads=num_heads[1], window_size=window_size,
                shift_size=0 if (i % 2 == 0) else window_size // 2)
            for i in range(depths[1])])

        self.block3 = nn.Sequential(*[
            SwinTransformerLayer(
                dim=embed_dims[2], num_heads=num_heads[2], window_size=window_size,
                shift_size=0 if (i % 2 == 0) else window_size // 2)
            for i in range(depths[2])])

        self.block4 = nn.Sequential(*[
            SwinTransformerLayer(
                dim=embed_dims[3], num_heads=num_heads[3], window_size=window_size,
                shift_size=0 if (i % 2 == 0) else window_size // 2)
            for i in range(depths[3])])
        self.norm = nn.BatchNorm2d(embed_dims[-1])

        self.Conv1 = nn.Conv2d(embed_dims[0], embed_dims[0], kernel_size=3, stride=1, padding=1, bias=False)
        self.Conv2 = nn.Conv2d(embed_dims[1], embed_dims[1], kernel_size=3, stride=1, padding=1, bias=False)
        self.Conv3 = nn.Conv2d(embed_dims[2], embed_dims[2], kernel_size=3, stride=1, padding=1, bias=False)
        self.Conv4 = nn.Conv2d(embed_dims[3], embed_dims[3], kernel_size=3, stride=1, padding=1, bias=False)

        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=.02)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)
        elif isinstance(m, nn.Conv2d):
            fan_out = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
            fan_out //= m.groups
            m.weight.data.normal_(0, math.sqrt(2.0 / fan_out))
            if m.bias is not None:
                m.bias.data.zero_()

    def reset_drop_path(self, drop_path_rate):
        dpr = [x.item() for x in torch.linspace(0, drop_path_rate, sum(self.depths))]
        cur = 0
        for i in range(self.depths[0]):
            self.block1[i].drop_path.drop_prob = dpr[cur + i]

        cur += self.depths[0]
        for i in range(self.depths[1]):
            self.block2[i].drop_path.drop_prob = dpr[cur + i]

        cur += self.depths[1]
        for i in range(self.depths[2]):
            self.block3[i].drop_path.drop_prob = dpr[cur + i]

        cur += self.depths[2]
        for i in range(self.depths[3]):
            self.block4[i].drop_path.drop_prob = dpr[cur + i]

    def forward_features(self, x):
        B = x.shape[0]
        outs = []

        # stage 1
        # Conv branch
        conv1 = self.Conv_stem1(x)

        # Transformer branch
        # B,C,W,H
        x1, H1, W1 = self.patch_embed1(x)
        Tran1 = self.block1(x1)

        #stage1_input:(1,3,512,1024), conv1:(1,64,128,256),Tran1:(1,64,128,256)
        y1 = conv1 + Tran1
        y1 = self.Conv1(y1)
        outs.append(y1)

        # stage 2
        # Conv branch
        conv2 = self.Conv_stem2(y1)
        # Transformer branch
        x2, H2, W2 = self.patch_embed2(y1)
        Tran2 = self.block2(x2)

        # stage2_input:(1,64,128,256), conv2:(1,128,64,128),Tran2:(1,128,64,128)
        y2 = conv2 + Tran2
        y2 = self.Conv2(y2)
        outs.append(y2)

        
        # stage 3
        # Conv branch
        conv3 = self.Conv_stem3(y2)
        # Transformer branch
        x3, H3, W3 = self.patch_embed3(y2)
        Tran3 = self.block3(x3)

        # stage3_input:(1,128,64,128), conv3:(1,320,32,64),Tran3:(1,320,32,64)
        y3 = conv3 + Tran3
        y3 = self.Conv3(y3)
        outs.append(y3)


        # stage 4
        # Conv branch
        conv4 = self.Conv_stem4(y3)
        # Transformer branch
        x4, H4, W4 = self.patch_embed4(y3)
        x4 = self.block4(x4)
        Tran4 = self.norm(x4)

        # stage4_input:(1,320,32,64), conv4:(1,512,16,32),Tran4:(1,512,16,32)
        y4 = conv4 + Tran4
        y4 = self.Conv4(y4)
        outs.append(y4)

        return outs

    def forward(self, x):
        x = self.forward_features(x)
        return x
    
class Tenc(EncoderTransformer):
    def __init__(self, **kwargs):
        super(Tenc, self).__init__(
            embed_dims=[96, 192, 384, 768], num_heads=[3, 6, 12, 24], depths=[2, 2, 6, 2],window_size=7
            )


In [15]:
class convprojection_base(nn.Module):
    def __init__(self, path=None, **kwargs):
        super(convprojection_base,self).__init__()

        # self.convd32x = UpsampleConvLayer(512, 512, kernel_size=4, stride=2)
        self.convd16x = UpsampleConvLayer(768, 384, kernel_size=4, stride=2)
        self.dense_4 = nn.Sequential(ResidualBlock(384))
        self.convd8x = UpsampleConvLayer(384, 192, kernel_size=4, stride=2)
        self.dense_3 = nn.Sequential(ResidualBlock(192))
        self.convd4x = UpsampleConvLayer(192, 96, kernel_size=4, stride=2)
        self.dense_2 = nn.Sequential(ResidualBlock(96))
        self.convd2x = UpsampleConvLayer(96, 24, kernel_size=4, stride=2)
        self.dense_1 = nn.Sequential(ResidualBlock(24))
        self.convd1x = UpsampleConvLayer(24, 12, kernel_size=4, stride=2)
        self.conv_output = ConvLayer(12, 3, kernel_size=3, stride=1, padding=1)

        self.active = nn.Tanh()        

    def forward(self,x1):
        res16x = self.convd16x(x1[3])

        if x1[2].shape[3] != res16x.shape[3] and x1[2].shape[2] != res16x.shape[2]:
            p2d = (0,-1,0,-1)
            res16x = F.pad(res16x,p2d,"constant",0)
        elif x1[2].shape[3] != res16x.shape[3] and x1[2].shape[2] == res16x.shape[2]:
            p2d = (0,-1,0,0)
            res16x = F.pad(res16x,p2d,"constant",0)
        elif x1[2].shape[3] == res16x.shape[3] and x1[2].shape[2] != res16x.shape[2]:
            p2d = (0,0,0,-1)
            res16x = F.pad(res16x,p2d,"constant",0)

        res8x = self.dense_4(res16x) + x1[2]

        res8x = self.convd8x(res8x)
        res4x = self.dense_3(res8x) + x1[1]
        res4x = self.convd4x(res4x)
        res2x = self.dense_2(res4x) + x1[0]
        res2x = self.convd2x(res2x)
        x = res2x
        x = self.dense_1(x)
        x = self.convd1x(x)

        return x

In [10]:
## The following is the network
class USCFormer(nn.Module):

    def __init__(self, output_nc=3, num_classes=19, seg_dim =32, **kwargs):
        super(USCFormer, self).__init__()

        self.Tenc = Tenc()
        
        self.convproj = convprojection_base()

        #self.clean = ConvLayer(8, 3, kernel_size=3, stride=1, padding=1)
        self.clean = ConvLayer(12, 3, kernel_size=3, stride=1, padding=1)

        #self.active = nn.Tanh()

        # semantic fused
        self.seg_convs = nn.Sequential(
            RDB(num_classes),
            nn.Conv2d(num_classes, seg_dim, 1, bias=True)
        )

        self.coarse_convs = nn.Sequential(
            Depthwise_separable_conv(output_nc, seg_dim)
        )

        self.combine_convs = nn.Sequential(
            nn.Conv2d(seg_dim, seg_dim, 3, 1, 1, bias=True),
            nn.Conv2d(seg_dim, seg_dim, 3, 1, 1, bias=True),
            nn.Conv2d(seg_dim, output_nc, 3, 1, 1, bias=True),
            nn.Tanh()
        )

    def forward(self, x, seg):

        #print(x.shape)
        #print(seg.shape)
        
        x1 = self.Tenc(x)

        x = self.convproj(x1)

        x = self.clean(x)

        # semantic fused
        f = self.coarse_convs(x) + self.seg_convs(seg)
        x1 = self.combine_convs(f)

        return x1

# Network 

In [45]:
# --- Define the network --- #
net = USCFormer()


# --- Define the seg network --- #
model_map = {
    'deeplabv3_resnet50': seg_network.deeplabv3_resnet50,
    'deeplabv3plus_resnet50': seg_network.deeplabv3plus_resnet50,
    'deeplabv3_resnet101': seg_network.deeplabv3_resnet101,
    'deeplabv3plus_resnet101': seg_network.deeplabv3plus_resnet101,
    'deeplabv3_mobilenet': seg_network.deeplabv3_mobilenet,
    'deeplabv3plus_mobilenet': seg_network.deeplabv3plus_mobilenet
}
num_classes = 19
model = model_map[models](num_classes=num_classes, output_stride=output_stride)
if separable_conv and 'plus' in model:
    model.convert_to_separable_conv(model.classifier)


# --- Multi-GPU --- #
net = net.to(device)
net = nn.DataParallel(net, device_ids=device_ids)


# --- Load the seg network weight --- #
checkpoint = torch.load(ckpt)
model.load_state_dict(checkpoint["model_state"])
model.eval()

# --- Multi-GPU --- #
model = model.to(device)
model = nn.DataParallel(model, device_ids=device_ids)

#summary(net,input_size=[(3, 672, 1120),(19, 672, 1120)])

# Training Setup

In [46]:
# --- Build optimizer --- #
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

# --- Define the L1 loss function --- #
l1_loss = nn.L1Loss()

# --- Define the ms_ssim loss function --- #
msssim_loss = MSSSIM()


# Previous Validation 

In [47]:
# --- Previous PSNR and SSIM in testing --- #
net.eval()

'''# load the lastest model
initial_epoch = findLastCheckpoint(save_dir='./{}'.format(exp_name))
if initial_epoch > 0:
    print('resuming by loading epoch %d' % initial_epoch)
    net.load_state_dict(torch.load(os.path.join('./{}'.format(exp_name), 'net_epoch%d.pth' % initial_epoch)))
'''

old_val_psnr, old_val_ssim, old_val_ciede = validation(net, model, val_data_loader, device, exp_name)

print('foggycityscapes old_val_psnr: {0:.2f}, old_val_ssim: {1:.4f}, old_val_ciede: {2:.4f}'.format(old_val_psnr, old_val_ssim, old_val_ciede))


foggycityscapes old_val_psnr: 13.20, old_val_ssim: 0.0624, old_val_ciede: 22.5058


# Training

In [48]:
#for epoch in range(initial_epoch, num_epochs):
for epoch in range(epoch_start, num_epochs):
    psnr_list = []
    start_time = time.time()
    adjust_learning_rate(optimizer, epoch)
#-------------------------------------------------------------------------------------------------------------
    for batch_id, train_data in enumerate(lbl_train_data_loader):

        input_image, gt = train_data
        input_image = input_image.to(device)
        gt = gt.to(device)

        # --- Zero the parameter gradients --- #
        optimizer.zero_grad()

        # --- Forward + Backward + Optimize --- #
        net.train()
        model.eval()

        seg = model(input_image).to(device)
        pred_image = net(input_image, seg)

        L1_loss = l1_loss(pred_image, gt)
        ms_ssim = -msssim_loss(pred_image, gt)

        cr_loss = CR.ContrastLoss()
        contrast_loss = cr_loss(pred_image, gt, input_image)

        loss = L1_loss  + alpha_loss * ms_ssim + beta_loss * contrast_loss

        loss.backward()
        optimizer.step()

        # --- To calculate average PSNR --- #
        psnr_list.extend(to_psnr(pred_image, gt))

        if not (batch_id % 100):
            print('Epoch: {0}, Iteration: {1}'.format(epoch+1, batch_id))

    # --- Calculate the average training PSNR in one epoch --- #
    train_psnr = sum(psnr_list) / len(psnr_list)

    # --- Save the network parameters --- #
    torch.save(net.state_dict(), './{}/latest'.format(exp_name))
    #torch.save(net.state_dict(), './{}/net_epoch{}.pth'.format(exp_name, str(epoch + 1)))
    
    # --- Use the evaluation model in testing --- #
    net.eval()

    val_psnr, val_ssim, val_ciede = validation(net, model, val_data_loader, device, exp_name)

    one_epoch_time = time.time() - start_time

    print("foggycityscapes")
    print_log(epoch+1, num_epochs, one_epoch_time, train_psnr, val_psnr, val_ssim, val_ciede, exp_name)

    # --- update the network weight --- #
    if val_psnr >= old_val_psnr:
        torch.save(net.state_dict(), './{}/best'.format(exp_name))
        print('model saved')
        old_val_psnr = val_psnr


Learning rate sets to 0.0002.


/home/yan/anaconda3/envs/xjm/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/yan/anaconda3/envs/xjm/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch: 1, Iteration: 0
Epoch: 1, Iteration: 100
Epoch: 1, Iteration: 200
Epoch: 1, Iteration: 300
Epoch: 1, Iteration: 400
Epoch: 1, Iteration: 500
Epoch: 1, Iteration: 600
Epoch: 1, Iteration: 700
Epoch: 1, Iteration: 800
Epoch: 1, Iteration: 900
Epoch: 1, Iteration: 1000
Epoch: 1, Iteration: 1100
Epoch: 1, Iteration: 1200
Epoch: 1, Iteration: 1300
Epoch: 1, Iteration: 1400
Epoch: 1, Iteration: 1500
Epoch: 1, Iteration: 1600
Epoch: 1, Iteration: 1700
Epoch: 1, Iteration: 1800
Epoch: 1, Iteration: 1900
Epoch: 1, Iteration: 2000
Epoch: 1, Iteration: 2100
Epoch: 1, Iteration: 2200
foggycityscapes
(2166s) Epoch [1/200], Train_PSNR:22.52, Val_PSNR:26.52, Val_SSIM:0.9040, Val_CIEDE:6.2823
model saved
Learning rate sets to 0.0002.
Epoch: 2, Iteration: 0
Epoch: 2, Iteration: 100
Epoch: 2, Iteration: 200
Epoch: 2, Iteration: 300
Epoch: 2, Iteration: 400
Epoch: 2, Iteration: 500
Epoch: 2, Iteration: 600
Epoch: 2, Iteration: 700
Epoch: 2, Iteration: 800
Epoch: 2, Iteration: 900
Epoch: 2, Iterati

KeyboardInterrupt: 